In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# SymPy helper functions

The {mod}`ampform.sympy` module contains a few classes that make it easier to construct larger expressions that consist of several mathematical definitions.

## Unevaluated expressions

The {func}`.unevaluated_expression` decorator makes it easier to write classes that represent a mathematical function definition. It makes a class that derives from {class}`sp.Expr <sympy.core.expr.Expr>` behave more like a {func}`~.dataclasses.dataclass` (see [PEP&nbsp;861](https://peps.python.org/pep-0681)). All you have to do is:

1. Specify the arguments the function requires.
2. Specify how to render the 'unevaluated' or 'folded' form of the expression with a `_latex_repr_` string or method.
3. Specify how to unfold the expression using an `evaluate()` method.

In the example below, we define a phase space factor $\rho^\text{CM}$ using the Chew-Mandelstam function (see PDG Resonances section, [Eq.&nbsp;(50.44)](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=15)). For this, you need to define a break-up momentum $q$ as well.

In [ ]:
import sympy as sp

from ampform.sympy import unevaluated_expression


@unevaluated_expression(real=False)
class PhspFactorSWave(sp.Expr):
    s: sp.Symbol
    m1: sp.Symbol
    m2: sp.Symbol
    _latex_repr_ = R"\rho^\text{{CM}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        cm = (
            (2 * q / sp.sqrt(s))
            * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
            - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
        ) / (16 * sp.pi**2)
        return 16 * sp.pi * sp.I * cm


@unevaluated_expression(real=False)
class BreakupMomentum(sp.Expr):
    s: sp.Symbol
    m1: sp.Symbol
    m2: sp.Symbol
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt((s - (m1 + m2) ** 2) * (s - (m1 - m2) ** 2) / (s * 4))

As can be seen, the LaTeX rendering of these classes makes them ideal for mathematically defining and building up larger amplitude models:

In [ ]:
from IPython.display import Math

from ampform.io import aslatex

s, m1, m2 = sp.symbols("s m1 m2")
q_expr = BreakupMomentum(s, m1, m2)
rho_expr = PhspFactorSWave(s, m1, m2)
Math(aslatex({e: e.evaluate() for e in [rho_expr, q_expr]}))

## Summations

The {class}`.PoolSum` class makes it possible to write sums over non-integer ranges. This is for instance useful when summing over allowed helicities. Here are some examples:

In [ ]:
from ampform.sympy import PoolSum

i, j, m, n = sp.symbols("i j m n")
expr = PoolSum(i**m + j**n, (i, (-1, 0, +1)), (j, (2, 4, 5)))
Math(aslatex({expr: expr.doit()}))

In [ ]:
import numpy as np

A = sp.IndexedBase("A")
λ, μ = sp.symbols("lambda mu")
to_range = lambda a, b: tuple(sp.Rational(i) for i in np.arange(a, b + 0.5))
expr = abs(PoolSum(A[λ, μ], (λ, to_range(-0.5, +0.5)), (μ, to_range(-1, +1)))) ** 2
Math(aslatex({expr: expr.doit()}))